# SQL query from table names - Continued

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')


## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
-- Creating tables with sample data
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(100)
);

INSERT INTO employees (ID_usr, name) VALUES
(1, 'Alice Johnson'),
(2, 'Bob Smith'),
(3, 'Charlie Davis');

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2024-01-01', 70000.00),
(2, '2024-01-01', 85000.00),
(3, '2024-01-01', 60000.00);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(255),
    Years DATE,
    Speciality VARCHAR(255),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 3, 'Harvard University', '2015-01-01', 'Computer Science'),
(2, 2, 2, 'MIT', '2016-01-01', 'Data Science'),
(3, 3, 1, 'Stanford University', '2017-01-01', 'Business Administration');

-- Few-Shot Sample Queries
-- Retrieve all employees and their salaries
-- Input: "Get the names and salaries of all employees."
SELECT employees.name, salary.salary, salary.year 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr;

-- Find employees with the highest salary in a given year
-- Input: "Who had the highest salary in 2024?"
SELECT employees.name, salary.salary 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr 
WHERE salary.year = '2024-01-01' 
ORDER BY salary.salary DESC 
LIMIT 1;

-- Get educational background of employees
-- Input: "Show the institution and speciality of each employee."
SELECT employees.name, studies.Institution, studies.Speciality 
FROM employees 
JOIN studies ON employees.ID_usr = studies.ID_usr;

"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

Apologies for the oversight. Please provide the questions you would like to ask about the provided tables, and I will be happy to assist you with the queries.


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

SELECT * 
FROM employees;
```
This SQL query selects all data from the "employees" table.


In [10]:
#new
print(return_CCRMSQL("Show the count of employees at each educational level", context_user))

```sql
SELECT studies.educational_level, COUNT(employees.ID_usr) AS employee_count
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr
GROUP BY studies.educational_level;
```


In [11]:
#old
print(return_CCRMSQL("Show the count of employees at each educational level", old_context_user))

This is your SQL:
```sql
SELECT educational_level, COUNT(ID_usr) AS employee_count
FROM studies
GROUP BY educational_level;
```

This SQL query selects the educational level from the "studies" table and counts the number of employees for each educational level by grouping the results based on the educational level.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [12]:
context_user = context.copy()
print(return_CCRMSQL("Find the employee who earns the most, including their salary and year.", context_user))


```sql
SELECT employees.name, salary.salary, salary.year
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
ORDER BY salary.salary DESC
LIMIT 1;
```


In [13]:
context_user = context.copy()
print(return_CCRMSQL("Who had the highest salary in 2024?", context_user))


```sql
SELECT employees.name, salary.salary 
FROM employees 
JOIN salary ON employees.ID_usr = salary.ID_usr 
WHERE salary.year = '2024-01-01' 
ORDER BY salary.salary DESC 
LIMIT 1;
```


In [14]:
context_user = context.copy()
print(return_CCRMSQL("Get the highest salary employee.", context_user))


```sql
SELECT employees.name, MAX(salary.salary) AS highest_salary
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
```
